## S-DES Python 3 Implementation
Implemented via iPython Notebook system. Can be run in regular Python interpreter by simple copying code in cells "In \[.\]"<br>
Keys and messages are represented as lists of 0 or 1 integers of corresponding length

Version 1.1

In [31]:
#Applies permutation p to list v
def applyP(v, p):
    return list(v[p[i] - 1] for i in range(0, len(p)))

#Cycled left shrink of list v for pos_num positions
def shl(v, pos_num):
    #eliminating negative and too large pos_num values
    pos_num = pos_num % len(v)
    pos_num = (pos_num + len(v)) % len(v)
    res = list(v[i] for i in range(pos_num, len(v)))
    res.extend(list(v[i] for i in range(0, pos_num)))
    return res

#Cycled right shrink of list v for pos_num positions
def shr(v, pos_num):
    #eliminating negative and too large pos_num values
    pos_num = pos_num % len(v)
    pos_num = (pos_num + len(v)) % len(v)
    res = list(v[i] for i in range(len(v) - pos_num, len(v)))
    res.extend(list(v[i] for i in range(0, len(v) - pos_num)))
    return res

#Generates round keys for S-DES algorithm
def genRoundKeys(basicKey):
    pkey = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
    permuted_key = applyP(basicKey, pkey)
    round_keys = []
    left = list(permuted_key[i] for i in range(0, 5))
    right = list(permuted_key[i] for i in range(5, 10))
    p_zip = [6, 3, 7, 4, 8, 5, 10, 9]
    left = list(shl(left, 1))
    right = list(shl(right, 1))
    key1 = list(left)
    key1.extend(right)
    round_keys.append(applyP(key1, p_zip))
    left = shl(left, 2)
    right = shl(right, 2)
    key2 = left
    key2.extend(right)
    round_keys.append(applyP(key2, p_zip))
    return round_keys

In [32]:
#Main S-DES function
#Contains transforms on the right part of a message
#during one round
def f_sdes(left, right, key):
    extend_perm = [4, 1, 2, 3, 2, 3, 4, 1]
    #applying permutation for extention
    new_mes = applyP(right, extend_perm)
    #applying key
    keyd_mes = list(new_mes[i]^key[i] for i in range(0, len(key)))
    #incorrect values on lection were in:
    #in S0[3][3] and in S1[0][0] and in S0[2][3]
    S0 = [
        [[0, 1], [0, 0], [1, 1], [1, 0]],
        [[1, 1], [1, 0], [0, 1], [0, 0]],
        [[0, 0], [1, 0], [0, 1], [1, 1]],
        [[1, 1], [0, 1], [1, 1], [1, 0]]
    ];
    S1 = [
        [[0, 0], [0, 1], [1, 0], [1, 1]],
        [[1, 0], [0, 0], [0, 1], [1, 1]],
        [[1, 1], [0, 0], [1, 0], [0, 0]],
        [[1, 0], [1, 0], [0, 0], [1, 1]]
    ];
    after_sq_blocks = list(S0[keyd_mes[0]*2+keyd_mes[3]][keyd_mes[1]*2+keyd_mes[2]])
    after_sq_blocks.extend(S1[keyd_mes[4 + 0]*2+keyd_mes[4 + 3]][keyd_mes[4 + 1]*2+keyd_mes[4 + 2]])
    P4 = [2, 4, 3, 1]
    after_perm = applyP(after_sq_blocks, P4)
    return after_perm

In [33]:
#S-DES round transforms function
def sdes_round(message, key):
    left = message[:4]
    right = message[4:]
    after_f = f_sdes(left, right, key)
    after_xor = list(left[i]^after_f[i] for i in range(0, 4))
    new_mes = list(after_xor)
    new_mes.extend(right)
    return new_mes

In [34]:
#Applies S-DES transforms to message
#using pre-defined key order
def sdes_ready_keys(message, keys):
    start_p = [2, 6, 3, 1, 4, 8, 5, 7]
    end_p = [4, 1, 3, 5, 7, 2, 8, 6]
    mixed_mes = applyP(message, start_p)
    round1 = sdes_round(mixed_mes, keys[0])
    message_for_r2 = list(round1[4:8])
    message_for_r2.extend(round1[0:4])
    round2 = sdes_round(message_for_r2, keys[1])
    result = applyP(round2, end_p)
    return result

In [35]:
#Encrypts message using S-DES
def encrypt(message, key):
    keys = genRoundKeys(key)
    return sdes_ready_keys(message, keys)

In [36]:
#Decrypts message using S-DES
def decrypt(message, key):
    keys = genRoundKeys(key)
    keys.reverse()
    return sdes_ready_keys(message, keys)

In [37]:
#intuit example:
#key: [1, 0, 1, 1, 1, 0, 0, 1, 1, 0]
#message: [1, 1, 1, 1, 0, 0, 1, 0]
#encrypted text: [1, 1, 1, 0, 1, 0, 1, 1]
#example from university (cryptoprotocols discipline) lection:
#key: [0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
#message: [1, 0, 1, 0, 0, 0, 1, 0]
#encrypted text: [0, 0, 1, 1, 1, 0, 0, 0]
encrypt([1, 1, 1, 1, 0, 0, 1, 0], [1, 0, 1, 1, 1, 0, 0, 1, 1, 0])

[0, 1, 1, 1, 0, 1, 1, 1]

In [38]:
decrypt([0, 1, 1, 1, 0, 1, 1, 1], [1, 0, 1, 1, 1, 0, 0, 1, 1, 0])

[1, 1, 1, 1, 0, 0, 1, 0]

In [39]:
#checking of correctness
SDES_correct = True
for i in range(0, 2**8):
    message = []
    for j in range(0, 8):
        message.append(1 if (i & (1<<j)) else 0)
    message.reverse()
    for j in range(0, 2**10):
        key = []
        for k in range(0, 10):
            key.append(1 if (j & (1<<k)) else 0)
        encrypted = encrypt(message, key)
        decrypted = decrypt(encrypted, key)
        #print('message')
        #print(message)
        #print('encrypted')
        #print(encrypted)
        #print('decrypted')
        #print(decrypted)
        result = True
        for k in range(0, 8):
            if(decrypted[k]!=message[k]):
                result=False
                break
        if(not result):
            SDES_correct = False
            break
    if(not SDES_correct):
        break
    #print(message)
print('SDES fine' if SDES_correct else 'Some decprypted data does not match with original')

SDES fine
